# Portfolio Optimization using Deep Reinforcement Learning
---

## 4.0 Feature Engineering and Data Preprocessing
---
We perform feature engineering and data preprocessing by:
* Adding Technical Indicators to the data. The technical inicators are used as inputs in the training of our Reinforcement Learning Model
* Adding Coveriance Matrices which is also used as input for training the Models
* Splitting the data into the training set and the testing (trading) set

### 4.1 Import Relevant Libraries

In [145]:
pip install finrl

  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /private/var/folders/3z/6b2hp23s5sg2j9x4jt4hyh_80000gn/T/pip-install-hj9vujzm/elegantrl_b705030d9ef74c6d81a8af4aca34c850
  Running command git clone --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /private/var/folders/3z/6b2hp23s5sg2j9x4jt4hyh_80000gn/T/pip-install-hj9vujzm/elegantrl_b705030d9ef74c6d81a8af4aca34c850
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit a531d493b710a872a4943532872846e00ad96abf
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [146]:
import pandas as pd
import numpy as np
import ta
from ta import add_all_ta_features
from ta.utils import dropna
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.meta.preprocessor.preprocessors import FeatureEngineer

### 4.2 Load the Data from the csv Files

In [147]:
# Load the whole data set
data = pd.read_csv('./datasets/data.csv')

# Load the close prices dataset
prices_data = pd.read_csv('./datasets/close_prices.csv')

In [148]:
pip install pickleshare


Note: you may need to restart the kernel to use updated packages.


In [149]:
%store 

Stored variables and their in-db values:
df_close_full_stocks             ->             date   HCLTECH.NS  EICHERMOT.NS  HINDA
filtered_stocks                  -> Index(['ITC.NS', 'NTPC.NS', 'HDFCBANK.NS', 'HINDUN


In [150]:
%store -r filtered_stocks

In [151]:
list_of_stocks = filtered_stocks
print(list_of_stocks)

Index(['ITC.NS', 'NTPC.NS', 'HDFCBANK.NS', 'HINDUNILVR.NS', 'CIPLA.NS',
       'GRASIM.NS', 'LT.NS', 'ASIANPAINT.NS', 'MARUTI.NS', 'RELIANCE.NS',
       'POWERGRID.NS', 'SUNPHARMA.NS', 'WIPRO.NS', 'TCS.NS', 'DRREDDY.NS',
       'INFY.NS', 'GAIL.NS', 'SBIN.NS', 'ICICIBANK.NS', 'HEROMOTOCO.NS'],
      dtype='object', name='stock_name')


In [152]:
data.head()

,date,tic,close,high,low,open,volume
0,2008-01-01,HDFCBANK.NS,173.899994,170.500000,172.949997,173.000000,1310520
1,2008-01-02,HDFCBANK.NS,173.479996,169.029999,171.529999,173.399994,6239750
2,2008-01-03,HDFCBANK.NS,171.000000,167.619995,169.440002,171.000000,7018640
3,2008-01-04,HDFCBANK.NS,171.399994,169.399994,169.639999,170.300003,3729820
4,2008-01-07,HDFCBANK.NS,167.839996,161.229996,165.539993,165.100006,6322460


In [153]:
data = data[data['tic'].isin(list_of_stocks)]

In [154]:
data.tic.unique()

array(['HDFCBANK.NS', 'HEROMOTOCO.NS', 'HINDUNILVR.NS', 'GRASIM.NS',
       'LT.NS', 'POWERGRID.NS', 'SBIN.NS', 'WIPRO.NS', 'INFY.NS',
       'CIPLA.NS', 'MARUTI.NS', 'TCS.NS', 'ICICIBANK.NS', 'RELIANCE.NS',
       'ASIANPAINT.NS', 'DRREDDY.NS', 'GAIL.NS', 'NTPC.NS', 'ITC.NS',
       'SUNPHARMA.NS'], dtype=object)

### 4.3 Add Technical Indicators
---
We define a function to add technical indicators to the dataset by making use of the ta library

The following indicators are considered:
* Volatility Average True Range (ATR)
* Volatility Bollinger Band Width (BBW)
* Volume On-balance Volume (OBV)
* Volume Chaikin Money Flow (CMF)
* Trend Moving Average Convergence Divergence (MACD)
* Trend Average Directional Index (ADX)
* Trend Fast Simple Moving Average (SMA)
* Trend Fast Exponential Moving Average (EMA)
* Trend Commodity Channel Index (CCI)
* Momentum Relative Strength Index (RSI)

In [155]:
# Define a Function for adding technical indicators

def add_features(data, feature_list, short_names):
    """
    Function to add technical indicators for features
    -Takes in a dataset with Open, High, Low, Close and Volume
    -Also takes in a list of the technical indicators to be added 
     as well as a list of the shortened indicator names
    """
    
    # list of column names to filter the features
    data_col_names = list(data.columns)
    filter_names = data_col_names + feature_list
    col_rename = data_col_names +  short_names
    
    # Add technical indicators using the ta Library
    data = add_all_ta_features(data, open="open", high="high", 
    low="low", close="close", volume="volume") 
    
    # Filter the Indicators with the required features
    data = data[filter_names]
    data.columns = col_rename # rename the columns to use shortened indicator names
    data = data.dropna()
    
    return data

In [156]:
# List of Features to add
feature_list= ['volatility_atr','volatility_bbw','volume_obv','volume_cmf',
               'trend_macd', 'trend_adx', 'trend_sma_fast', 
               'trend_ema_fast', 'trend_cci', 'momentum_rsi']

# Short names of the features
short_names = ['atr', 'bbw','obv','cmf','macd', 'adx', 'sma', 'ema', 'cci', 'rsi']

#feature_list= ['volatility_atr','volatility_bbw','volume_obv','volume_cmf','trend_macd']

# Short names of the features
#short_names = ['atr', 'bbw','obv','cmf','macd']

In [157]:
# Add Indicators to our dataset
data_with_features = data.copy()
data_with_features = add_features(data_with_features, feature_list, short_names)

In [158]:
data_with_features.head()

,date,tic,close,high,low,open,volume,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi
25,2008-02-05,HDFCBANK.NS,159.740005,150.024994,151.225006,156.000000,6071410,12.075972,20.379451,-33513450,-5.374176,-3.754521,0.000000,163.182500,163.078783,-62.083845,42.717502
26,2008-02-06,HDFCBANK.NS,151.475006,145.139999,150.009995,149.990005,7865940,12.328375,21.666539,-41379390,-5.400246,-4.402066,0.000000,162.555417,161.293586,-95.750705,36.650614
27,2008-02-07,HDFCBANK.NS,151.199997,148.199997,149.179993,149.899994,16715320,11.423039,22.065631,-58094710,-5.135270,-4.881173,13.850025,160.322084,159.740726,-87.093374,36.465036
28,2008-02-08,HDFCBANK.NS,150.300003,142.300003,144.195007,149.100006,3880660,11.170734,22.464942,-61975370,-5.276904,-5.272711,13.135812,158.680417,158.288307,-123.495973,35.825719
29,2008-02-11,HDFCBANK.NS,146.899994,135.070007,141.074997,144.195007,6926890,11.576660,21.914463,-68902260,-5.386688,-5.790609,12.899149,157.280417,156.536259,-180.043190,33.440450


In [159]:
feature_list = list(data_with_features.columns)[7:]

In [160]:
print(feature_list)

['atr', 'bbw', 'obv', 'cmf', 'macd', 'adx', 'sma', 'ema', 'cci', 'rsi']


### 4.4 Add Covariance Matrix
---
We define a function that will add Covarance Matrices to our dataset

In [161]:
def add_cov_matrix(df):
    """
    Function to add Coveriance Matrices as part of the defined states
    """
    # Sort the data and index by date and tic
    df=df.sort_values(['date','tic'],ignore_index=True) 
    df.index = df.date.factorize()[0]
    
    cov_list = [] # create empty list for storing coveriance matrices at each time step
    
    # look back for constructing the coveriance matrix is one year
    lookback=252
    for i in range(lookback,len(df.index.unique())):
        data_lookback = df.loc[i-lookback:i,:]
        price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values 
        covs = covs#/covs.max()
        cov_list.append(covs)
        
    df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
    df = df.merge(df_cov, on='date')
    df = df.sort_values(['date','tic']).reset_index(drop=True)
    
    return df

In [162]:
# Add Covariance Matrices to our dataset
data_with_features_covs = data_with_features.copy()
data_with_features_covs = add_cov_matrix(data_with_features_covs)

/var/folders/3z/6b2hp23s5sg2j9x4jt4hyh_80000gn/T/ipykernel_54551/661913900.py:16: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return_lookback = price_lookback.pct_change().dropna()
/var/folders/3z/6b2hp23s5sg2j9x4jt4hyh_80000gn/T/ipykernel_54551/661913900.py:16: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return_lookback = price_lookback.pct_change().dropna()
/var/folders/3z/6b2hp23s5sg2j9x4jt4hyh_80000gn/T/ipykernel_54551/661913900.py:16: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-l

In [163]:
data_with_features_covs.head()

,date,tic,close,high,low,open,volume,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi,cov_list
0,2009-01-13,ASIANPAINT.NS,91.699997,88.500000,91.235001,88.500000,65800,2.709892,8.344600,25599954665,-9.929696,-0.321724,8.273333,89.891666,90.273798,107.976227,52.883757,"[[0.0005821350723573744, 0.0001385649017777150..."
1,2009-01-13,CIPLA.NS,189.649994,184.000000,185.350006,185.000000,901712,8.145647,10.658031,17931436408,-5.061957,0.163100,8.062326,190.170832,191.058647,12.530568,48.233742,"[[0.0005821350723573744, 0.0001385649017777150..."
2,2009-01-13,DRREDDY.NS,478.000000,448.000000,452.750000,465.750000,544994,24.985094,9.716542,26047694828,-5.436297,5.426979,21.699377,482.574999,482.291917,-46.397095,50.524243,"[[0.0005821350723573744, 0.0001385649017777150..."
3,2009-01-13,GAIL.NS,39.375019,37.875019,38.756268,38.606270,9334277,2.253717,15.414591,26198304659,-4.217676,-0.335640,8.487785,39.254707,39.600802,-24.680875,47.371823,"[[0.0005821350723573744, 0.0001385649017777150..."
4,2009-01-13,GRASIM.NS,209.852203,202.908554,204.891357,205.570282,1994905,9.809186,22.975549,1917389030,-3.682218,9.910951,38.354100,200.315626,202.191527,125.899067,65.313483,"[[0.0005821350723573744, 0.0001385649017777150..."


In [164]:
# Number of features before adding covariance matrices
num_features_before = len(data_with_features.columns)

# Number of features after adding covariance matrices
num_features_after = len(data_with_features_covs.columns)

# Check if the number of features before and after are the same
print("Number of features before adding covariance matrices:", num_features_before)
print("Number of features after adding covariance matrices:", num_features_after)


Number of features before adding covariance matrices: 17
Number of features after adding covariance matrices: 18


### 4.6 Store the Dataframe

In [165]:
df = data_with_features_covs

In [166]:
%store df

Stored 'df' (DataFrame)


In [167]:
%store

Stored variables and their in-db values:
df                               ->              date            tic        close     
df_close_full_stocks             ->             date   HCLTECH.NS  EICHERMOT.NS  HINDA
filtered_stocks                  -> Index(['ITC.NS', 'NTPC.NS', 'HDFCBANK.NS', 'HINDUN
